# Text Generation Referring to  Local Document with Transformers Pipeline, LangChain and Chroma

In this notebook, we will use a large language model (LLM) to perform text generation to answer questions based on local stored PDF document.

## Preparation

Install python packages.

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.7 MB/s eta 0:00:00


In [2]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 6.7 MB/s eta 0:00:00


In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=70f71d0caa144c5f1866978033ce7279973f730aed37d722e70420b37711fa33
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.7/966.7 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s

In [5]:
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 106.5 MB/s eta 0:00:00


In [6]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.9 MB/s eta 0:00:00


Check GPU information.

In [7]:
!nvidia-smi

Mon Jul 17 07:54:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Fix random number seed.

In [8]:
import random
import numpy as np
import torch

def set_seed(seed=0):
  # for Python
  random.seed(seed)

  # for NumPy
  np.random.seed(seed)

  # for PyTorch, CUDA and cuDNN
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

In [9]:
set_seed()

Get device.

In [10]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
  device_id = 0
else:
  device = torch.device('cpu')
  device_id = -1

print(device)
print(device_id)

cuda
0


## Get PDF Document

As an example, we use the following PDF document describing the Manifesto for Agile Software Development by Information-technology Promotion Agency, Japan (IPA).
* https://www.ipa.go.jp/jinzai/skill-standard/plus-it-ui/itssplus/ps6vr70000001i7c-att/000065601.pdf

In [11]:
!wget https://www.ipa.go.jp/jinzai/skill-standard/plus-it-ui/itssplus/ps6vr70000001i7c-att/000065601.pdf

--2023-07-17 07:54:05--  https://www.ipa.go.jp/jinzai/skill-standard/plus-it-ui/itssplus/ps6vr70000001i7c-att/000065601.pdf
Resolving www.ipa.go.jp (www.ipa.go.jp)... 13.35.35.92, 13.35.35.31, 13.35.35.28, ...
Connecting to www.ipa.go.jp (www.ipa.go.jp)|13.35.35.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2019698 (1.9M) [application/pdf]
Saving to: ‘000065601.pdf’

000065601.pdf       100%[===================>]   1.93M  8.16MB/s    in 0.2s    

2023-07-17 07:54:06 (8.16 MB/s) - ‘000065601.pdf’ saved [2019698/2019698]



In [12]:
from langchain.document_loaders import PDFMinerLoader

file = '000065601.pdf'

loader = PDFMinerLoader(file)
text = loader.load()
pages = text[0].page_content.split('\x0c')

In [13]:
len(pages)

22

In [14]:
pages[0]

'アジャイル領域へのスキル変革の指針\n\nアジャイルソフトウェア開発宣言の\n読みとき方\n\n2020年2月\n\nITSS+ ／ アジャイル領域へのスキル変革の指針\n\nAll Rights Reserved Copyright© IPA 2018-2020\n\n'

## Construct Vector Database

Prepare an embedding.

Here, for example, we use OpenCALM-1B (https://huggingface.co/cyberagent/open-calm-1b), which was pre-trained on Japanese datasets by CyberAgent, Inc.

In [15]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

model_name = 'cyberagent/open-calm-1b'
embeddings = HuggingFaceEmbeddings(model_name=model_name)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/cyberagent_open-calm-1b were not used when initializing GPTNeoXModel: ['embed_out.weight']
- This IS expected if you are initializing GPTNeoXModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPTNeoXModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Convert to embedded vector for each page and store in vector DB.

In [16]:
import chromadb
from chromadb.config import Settings

settings = Settings(chroma_db_impl='duckdb+parquet', persist_directory='./db')
client = chromadb.Client(settings=settings)
client.reset()
collection = client.create_collection(name='collection')

for i, page in enumerate(pages, start=1):
  text = page.replace('\n', '')
  if text == '':
    continue
  embedded_docs = embeddings.embed_documents([text])
  collection.add(
      embeddings=embedded_docs,
      documents=[text],
      metadatas={'source': file},
      ids=f'id{i}'
  )

print(collection.count())

# Following code is needed to persist DB
# client.persist()

21


## Building a Transformers Pipeline

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers import pipeline

task='text-generation'

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

kwargs = {
    'max_new_tokens': 64,
    'do_sample': True,
    'temperature': 0.7,
    'top_p': 0.9,
    'repetition_penalty': 1.05,
    'pad_token_id': tokenizer.pad_token_id
}

pipe = pipeline(
    task=task,
    model=model,
    tokenizer=tokenizer,
    device=device_id,
    torch_dtype=torch.float16,
    **kwargs
)

## LangChain

In [18]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)

## Search the vector DB to extract candidates and pass them to LLMChain to get the answer

In [19]:
query = 'アジャイルソフトウェア開発宣言で、プロセスやツールよりも重視していることは？'

In [20]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = HuggingFacePipeline(pipeline=pipe)

template = """
以下の情報を活用して、与えられた質問に回答してください。

情報: {content}

質問: {query}

回答:
"""
prompt = PromptTemplate(template=template, input_variables=['content', 'query'])

llm_chain = LLMChain(llm=llm, prompt=prompt)

Search vector DB.

In [21]:
query = 'アジャイルソフトウェア開発宣言で、プロセスやツールよりも重視していることは？'
embedded_query = embeddings.embed_query(query)
results = collection.query(
    query_embeddings=embedded_query,
    n_results=5,
)

Pass above search results to LLMChain to get answers.

In [22]:
docs = [{'content': doc, 'query': query} for doc in results['documents'][0]]

In [23]:
llm_chain.apply(input_list=docs)

[{'text': '"アジェンダの作成と共有を習慣化する。"これは非常に重要ですね。「カンバン方式」「ペアプログラミング」、「モブプロ」、その他様々な方法がありますが、“今やっていることそのものを変えるのではなくて未来の変化に対応できるようにする。”という点がとても大切です。“変化に対応する”。これが一番大切だと思いますね。『スクラム』ではそれを“'},
 {'text': ' チーム全員による合意形成を重視して生産性を高めましょう'},
 {'text': '原文には記載がありませんが、『顧客の要求事項』の中で『組織的な成功への戦略的計画』(Strategic Plan of Success)という項目に注目しましょう。『継続的かつ効果的な仕事を行うためには』、組織は次の3つの段階を踏む必要があると述べられています。・ビジョンの共有 ・相互支援体制の構築及び維持および強化'},
 {'text': 'アジャイルは「俊敏な」という意味であり、「Agile(アジール)」という単語が語源です。「迅速かつ柔軟性高く動作するソフトウェアの開発」「変化への柔軟に対応する開発手法」、またその手法を実現する思想を表しています。[1] このキーワードにはいくつかの意味があり、・ プロダクトオーナーが決めたゴールを達成するためにチーム全体で'},
 {'text': 'Q1. AJAXアプリケーションの開発はどのように行われていますでしょうか?「AJXA」(Asynchronous JavaScript and XML Architecture, a JSON Application Framework - )というフレームワークがあります。「ajaxx (JavaScriptによるXML Webサービス構築のためのオープンソース・コード)」によって開発されたものです。(ただしJSON'}]